# SPXW Dataset

The spxw weekly dataset contains a snapshop of all the S&P500 Weekly options on the data Sep 3 2025.

In [1]:
from volkit.datasets import spxw

Below is the full dataset.


* ``K`` : float, strike
* ``F_bid``, ``F_ask``: underlying future bid/ask
* ``C_bid``, ``C_ask``, ``C_vol``, ``C_oi`` : call bid/ask/volume/open interst
* ``P_bid``, ``P_ask``, ``P_vol``, ``P_oi``  : put bid/ask/volume/open interest
* ``T`` : float, time to expiry in trading years (``D / 252``)
* ``D`` : int, calendar days to expiry (``ExpDate - Date``) 

In [2]:
df = spxw()
df

,quote_date,expiration_date,K,F_bid,F_ask,C_ask,P_ask,C_ask_size,P_ask_size,C_bid,P_bid,C_bid_size,P_bid_size,C_oi,P_oi,C_vol,P_vol,D,T
0,2019-06-26,2019-06-26,1700,2917.8,2918.42,1229.20,0.05,78,1699,1209.10,0.0,78,0,0,24,0,0,0,0.000000
1,2019-06-26,2019-06-26,1800,2917.8,2918.42,1129.20,0.05,78,5956,1109.10,0.0,78,0,0,4333,0,0,0,0.000000
2,2019-06-26,2019-06-26,1850,2917.8,2918.42,1079.20,0.05,78,7956,1059.10,0.0,78,0,0,11328,0,0,0,0.000000
3,2019-06-26,2019-06-26,1900,2917.8,2918.42,1029.20,0.05,79,5067,1009.10,0.0,79,0,20,3240,0,0,0,0.000000
4,2019-06-26,2019-06-26,1950,2917.8,2918.42,979.20,0.05,78,2084,959.10,0.0,78,0,0,382,0,0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5187,2019-06-26,2020-06-30,3800,2917.8,2918.42,2.70,866.90,149,100,2.35,850.6,124,100,0,0,0,0,370,1.468254
5188,2019-06-26,2020-06-30,3900,2917.8,2918.42,1.95,964.00,124,100,1.65,947.7,124,100,0,0,10,0,370,1.468254
5189,2019-06-26,2020-06-30,4000,2917.8,2918.42,1.55,1061.40,125,100,1.20,1045.1,125,100,0,0,0,0,370,1.468254
5190,2019-06-26,2020-06-30,4100,2917.8,2918.42,1.25,1159.00,125,100,0.85,1142.7,125,100,0,0,0,0,370,1.468254


The data loader has two arguments ``min_vol`` and ``D`` that allow you to load and filter subset.

Below we load only the options with a traded volume of at least 100 in both calls and puts, and with 7 days till expiration.

In [3]:
df = spxw(min_vol=100, D=7)
df

,quote_date,expiration_date,K,F_bid,F_ask,C_ask,P_ask,C_ask_size,P_ask_size,C_bid,P_bid,C_bid_size,P_bid_size,C_oi,P_oi,C_vol,P_vol,D,T
683,2019-06-26,2019-07-03,2900,2917.8,2918.42,38.0,19.1,11,40,37.6,18.7,14,20,1669,1859,330,333,7,0.027778
687,2019-06-26,2019-07-03,2920,2917.8,2918.42,25.7,26.8,23,14,25.4,26.4,8,14,230,321,444,240,7,0.027778
688,2019-06-26,2019-07-03,2925,2917.8,2918.42,23.0,29.1,23,14,22.7,28.7,11,14,1968,2941,755,908,7,0.027778
